In [ ]:
%%sql
DROP TABLE daily_product_revenue;

In [1]:
%%sql
CREATE TABLE order_count_by_status
AS
SELECT orders.order_status, count(*) AS order_count
FROM orders
GROUP BY 1;

In [2]:
%%sql
SELECT * FROM order_count_by_status

,order_status,order_count
0,COMPLETE,22899
1,ON_HOLD,3798
2,PENDING_PAYMENT,15030
3,PENDING,7610
4,CLOSED,7556
5,CANCELED,1428
6,PROCESSING,8275
7,PAYMENT_REVIEW,729
8,SUSPECTED_FRAUD,1558


In [8]:
%%sql
SELECT * FROM orders_stg;

,order_id,order_date,order_customer_id,order_status


In [ ]:
%%sql
CREATE TABLE orders_stg
AS
SELECT * FROM orders WHERE 1=2;

In [ ]:
%%sql
CREATE TABLE daily_revenue
AS
SELECT to_char(o.order_date, 'yyyy-mm-dd'),
       round(sum(oi.order_item_subtotal)::numeric, 2) AS order_revenue
FROM orders AS o
    JOIN order_items AS oi
        ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date;

In [24]:
%%sql
SELECT * FROM daily_revenue;

,to_char,order_revenue
0,2014-07-17,36384.77
1,2014-07-14,29937.52
2,2013-09-03,44379.10
3,2014-07-01,40165.66
4,2013-08-04,35093.01
...,...,...
359,2013-09-19,51600.34
360,2013-11-30,63914.68
361,2014-03-30,52406.98
362,2014-05-02,42595.32


In [ ]:
%%sql
CREATE TABLE daily_product_revenue
AS
SELECT o.order_date,
      oi.order_item_order_id,
      round(sum(oi.order_item_subtotal)::numeric, 2) AS order_revenue
FROM orders AS o
    JOIN order_items AS oi
        ON o.order_id = oi.order_item_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY 1, 2
ORDER BY round(sum(oi.order_item_subtotal)::numeric, 2) DESC;

In [42]:
%%sql
SELECT * FROM daily_product_revenue
WHERE order_revenue >= 300
  AND order_revenue <= 499
ORDER BY order_revenue DESC;

,order_id,order_item_order_id,order_revenue
0,20673,8274,399.98
1,2187,877,399.98
2,3231,1306,399.98
3,54050,21617,399.98
4,53167,21257,399.98
...,...,...,...
3618,41760,16711,325.00
3619,20666,8271,325.00
3620,31858,12734,325.00
3621,22154,8875,325.00


In [51]:
%%sql
SELECT to_char(dr.to_char::timestamp, 'yyyy-mm') AS order_month,
      sum(order_revenue) AS order_revenue
FROM daily_revenue AS dr
GROUP BY 1
ORDER BY 1;

,order_month,order_revenue
0,2013-07,333465.45
1,2013-08,1221828.90
2,2013-09,1302255.80
3,2013-10,1171686.92
4,2013-11,1379935.33
5,2013-12,1277719.60
6,2014-01,1230221.74
7,2014-02,1217770.09
8,2014-03,1271350.97
9,2014-04,1249723.52


In [ ]:
%%sql
ALTER TABLE daily_revenue
RENAME COLUMN to_char TO order_date;

In [58]:
%%sql
SELECT to_char(dr.order_date::timestamp, 'yyyy-mm') AS order_month,
      dr.order_date,
      dr.order_revenue,
      sum(order_revenue) OVER (
          PARTITION BY to_char(dr.order_date::timestamp, 'yyyy-mm')) AS monthly_order_revenue
FROM daily_revenue AS dr
ORDER BY 1;

,order_month,order_date,order_revenue,monthly_order_revenue
0,2013-07,2013-07-25,31547.23,333465.45
1,2013-07,2013-07-29,54579.70,333465.45
2,2013-07,2013-07-31,59212.49,333465.45
3,2013-07,2013-07-26,54713.23,333465.45
4,2013-07,2013-07-30,49329.29,333465.45
...,...,...,...,...
359,2014-07,2014-07-12,38449.77,955590.77
360,2014-07,2014-07-06,16451.76,955590.77
361,2014-07,2014-07-24,50885.19,955590.77
362,2014-07,2014-07-19,38420.99,955590.77


In [62]:
%%sql
SELECT dr.*,
      sum(order_revenue) OVER (
          PARTITION BY order_date
      ) AS total_revenue
FROM daily_revenue AS dr
ORDER BY total_revenue DESC;

,order_date,order_revenue,total_revenue
0,2013-11-03,69977.41,69977.41
1,2013-12-06,67137.97,67137.97
2,2013-09-26,66470.50,66470.50
3,2014-06-19,65047.41,65047.41
4,2014-04-08,64946.35,64946.35
...,...,...,...
359,2013-12-02,19278.83,19278.83
360,2013-08-13,17956.88,17956.88
361,2014-05-30,17890.91,17890.91
362,2014-07-06,16451.76,16451.76


In [75]:
%%sql
SELECT order_date,
      order_item_order_id,
      order_revenue,
      rank() OVER (ORDER BY order_revenue DESC ) AS rnk,
      dense_rank() OVER (ORDER BY order_revenue DESC ) drnk
FROM daily_product_revenue
WHERE to_char(order_date, 'yyyy-mm-dd') = '2014-01-01'
ORDER BY order_revenue DESC;

,order_date,order_item_order_id,order_revenue,rnk,drnk
0,2014-01-01 00:00:00.000000,10347,1039.93,1,1
1,2014-01-01 00:00:00.000000,10354,999.95,2,2
2,2014-01-01 00:00:00.000000,10352,839.94,3,3
3,2014-01-01 00:00:00.000000,10370,699.87,4,4
4,2014-01-01 00:00:00.000000,10374,549.98,5,5
5,2014-01-01 00:00:00.000000,10357,529.97,6,6
6,2014-01-01 00:00:00.000000,10369,499.98,7,7
7,2014-01-01 00:00:00.000000,10342,499.97,8,8
8,2014-01-01 00:00:00.000000,10367,457.84,9,9
9,2014-01-01 00:00:00.000000,24719,399.98,10,10


In [83]:
%%sql
SELECT to_char(order_date::timestamp, 'yyyy-mm-dd') AS date,
       order_item_order_id,
       order_revenue,
       rank() OVER (PARTITION BY order_date ORDER BY order_revenue DESC) AS rnk,
       dense_rank() OVER (PARTITION BY order_revenue ORDER BY order_revenue DESC) drnk
FROM daily_product_revenue
WHERE to_char(order_date, 'yyyy-mm') = '2014-01';

,date,order_item_order_id,order_revenue,rnk,drnk
0,2014-01-13,11105,1599.92,1,1
1,2014-01-07,10696,1499.87,1,1
2,2014-01-07,10699,1349.82,2,1
3,2014-01-04,10533,1239.88,1,1
4,2014-01-31,12307,1229.93,1,1
...,...,...,...,...,...
1511,2014-01-13,11112,27.99,51,1
1512,2014-01-30,12198,25.00,71,1
1513,2014-01-05,10548,19.98,65,1
1514,2014-01-09,27142,15.99,48,1


In [87]:
%%sql
SELECT * FROM (
    SELECT order_date,
           order_item_order_id,
           order_revenue,
           rank() OVER (ORDER BY order_revenue) AS rnk,
           dense_rank() OVER (ORDER BY order_revenue DESC ) as drnk
    FROM daily_product_revenue
    WHERE to_char(order_date, 'yyyy-mm-dd') = '2014-01-01'
    ORDER BY order_revenue DESC
) AS q
WHERE drnk <= 5
ORDER BY order_revenue DESC

,order_date,order_item_order_id,order_revenue,rnk,drnk
0,2014-01-01 00:00:00.000000,10347,1039.93,33,1
1,2014-01-01 00:00:00.000000,10354,999.95,32,2
2,2014-01-01 00:00:00.000000,10352,839.94,31,3
3,2014-01-01 00:00:00.000000,10370,699.87,30,4
4,2014-01-01 00:00:00.000000,10374,549.98,29,5


In [90]:
%%sql
SELECT to_char(order_date, 'yyyy-mm-dd'),
       order_item_order_id,
       order_revenue
FROM daily_product_revenue

,order_date,order_item_order_id,order_revenue
0,2014-05-11 00:00:00.000000,18735,1699.89
1,2014-01-13 00:00:00.000000,11105,1599.92
2,2014-02-19 00:00:00.000000,13621,1549.79
3,2014-07-08 00:00:00.000000,22030,1539.85
4,2014-07-14 00:00:00.000000,22365,1509.88
...,...,...,...
17831,2014-05-12 00:00:00.000000,18772,14.99
17832,2013-11-19 00:00:00.000000,7505,14.99
17833,2014-05-31 00:00:00.000000,26310,14.99
17834,2014-01-02 00:00:00.000000,10396,14.99


In [ ]:
from email.feedparser import headerRE

Type in your message headerRE